In [ ]:
!pip install git+https://github.com/jlopezalo/PyFMReader.git

In [ ]:
!pip install git+https://github.com/jlopezalo/PyFMRheo.git

In [ ]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import gmean, sem
import seaborn as sns
import matplotlib.pyplot as plt
from pyafmrheo.models.double_powerlaw import DoublePowerLawModel

In [ ]:
fluidity_exp_microrheo = []

dataframe = pd.read_csv("path_to_your_file_with_G")

for file_id in dataframe.file_id.unique():
    
    if file_id in discarded_files:
        continue
    
    subdf = dataframe[dataframe.file_id == file_id]
    procdf = subdf.loc[:, ['frequency', 'G_storage', 'G_loss']]
    aggdf = procdf.groupby(by='frequency', as_index=False).apply(geo_mean_overflow)
    
    # Define model
    model = DoublePowerLawModel()
    model.A_max = np.inf
    model.A_min = 0
    model.B_max = np.inf
    model.B_min = 0
    model.A_init = 1e3
    model.B_init = 1e3
    model.alpha_max = 1
    model.alpha_min = 0
    model.beta_max=1
    model.beta_min = 0
    model.alpha_init = 0.1
    model.beta_init = 0.7
    freq = aggdf.frequency.values
    
    split_indx = len(freq)
    freq_for_fit = np.r_[freq, freq]
    w0 = 1
    G = np.r_[aggdf.G_storage.values, aggdf.G_loss.values]
    model.fit(freq_for_fit, G, w0, split_indx)
    G_res = model.eval(freq_for_fit, w0, split_indx)
    
    # Plot
    print(file_id)
    print(model.alpha)
    print(model.beta)
    print(model.A)
    print(model.B)
    print(len(aggdf.G_storage.values))
    fluidity_exp_microrheo.append(model.alpha)
    sns.set_palette(['#E72E38', '#E72E38'])
    sns.lineplot(x='frequency', y='G_storage', data=procdf, marker='', linestyle='', estimator=geo_mean_overflow, errorbar='se', err_style='bars')
    sns.lineplot(x='frequency', y='G_loss', data=procdf, marker='', linestyle='', estimator=geo_mean_overflow, errorbar='se', err_style='bars')
    plt.plot(freq, G_res[:split_indx], color='#2B3E51',linewidth=2.5, linestyle='--')
    plt.plot(freq, G_res[split_indx:], color='#2B3E51',linewidth=2.5, linestyle='--')
    plt.plot(freq, aggdf.G_storage.values, 'o', color='#E72E38',)
    plt.plot(freq, aggdf.G_loss.values, 'o', color='#E72E38', fillstyle='none')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Frequency [Hz]', fontsize = 15)
    plt.ylabel("G'& G'' [Pa]", fontsize = 15)
    plt.show()